<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/6_EXTRA_Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FAISS library**

[Link to the manual](https://www.pinecone.io/learn/series/faiss/)

In [1]:
!pip install -U sentence-transformers
!pip install faiss-cpu --no-cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import requests
from io import StringIO

import pandas as pd
import numpy as np
import json

from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from sentence_transformers.util import dot_score
from sentence_transformers.util import cos_sim

import time
import faiss

## Dataset

The dataset is formed by data fetched from more urls having the same structure.

Be careful, the more you fetch the longer this takes in embedding them.

At the end the phrases to be embedded are fetched in the array *sentences*

In [8]:
!wget https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/refs/heads/main/data/500news.json
with open("500news.json", 'r') as json_file:
  articles = json.load(json_file)

--2025-02-20 15:31:44--  https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/refs/heads/main/data/500news.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147867 (144K) [text/plain]
Saving to: ‘500news.json.1’

500news.json.1      100%[===================>] 144.40K  --.-KB/s    in 0.01s   

2025-02-20 15:31:44 (11.6 MB/s) - ‘500news.json.1’ saved [147867/147867]



In [10]:
import nltk
tokenizer = nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [13]:
sentences = []
for art in articles:
  sent = nltk.sent_tokenize(art["maintext"])
  for s in sent:
    sentences.append(s)
len(sentences)

776

In [14]:
sentences[0]

'Victims are civilians: the attacker took his own life'

## Sentence embeddings

We build our dense vector representations of each sentence using some libraries that we list in the code, as options.

Other models in: https://sbert.net/docs/pretrained_models.html

We limit the number of sentences for time reasons.

In [15]:
## Other models:
##    model = SentenceTransformer('bert-base-nli-mean-tokens')
##    model = SentenceTransformer("hkunlp/instructor-large")

# Initialize sentence transformer model
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# create sentence embeddings - we limit to s sentences because of time reasons
s = 400
sentence_embeddings = model.encode(sentences[:s])

print(f"\n\nnumber of examples = {sentence_embeddings.shape[0]}, and number of dimensions = {sentence_embeddings.shape[1]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]



number of examples = 400, and number of dimensions = 384


## Preliminary test for query-phrase embedding similarity

Print for the first 10 sentences, their cosine similarity (in [0,1]), their dot-product similarity (positive), and part of their embedding vectors.

In [19]:
query = "I love sports"
query_embedding = model.encode(query)

# Prints only the results of the first 10 phrases
for i,v in enumerate(sentence_embeddings[:10]):
  print("Sentence: ", sentences[i])
  print("Similarity (dot_score):", float(dot_score(query_embedding, v)[0][0]))
  print("Similarity (cosine sim):", float(util.cos_sim(query_embedding, v)[0][0]))
  print("Embedding: ", sentence_embeddings[i][:10], "...\n") # So you can store them

Sentence:  Victims are civilians: the attacker took his own life
Similarity (dot_score): -0.05554763227701187
Similarity (cosine sim): -0.055547647178173065
Embedding:  [ 0.06090301  0.12335551  0.00399603 -0.06866315  0.12200678  0.03321851
  0.11034612  0.04387144  0.00760243  0.07002521] ...

Sentence:  The traditional party for lighting up the lights
Similarity (dot_score): 0.06056618690490723
Similarity (cosine sim): 0.06056618690490723
Embedding:  [ 0.01536698  0.04639235  0.02504889  0.00739377  0.00088897  0.01775539
  0.02517276 -0.06993458 -0.02468251  0.06536196] ...

Sentence:  250 events are planned throughout the country, one of the most impressive of recent years.
Similarity (dot_score): 0.11960402876138687
Similarity (cosine sim): 0.11960402876138687
Embedding:  [ 0.11990421  0.00178921  0.01813955 -0.02085359  0.04035327  0.04969554
 -0.09003521 -0.10944775 -0.02945603  0.1039343 ] ...

Sentence:  The strike against one of the cornerstones of Macron's programme: pensio

# **Brute Force SCAN**

### An example of finding the most cosine-similar phrase via TORCH and scanning


In [22]:
import torch

## The query sentence
print("QUERY:: {}".format(query))
query_embedding = model.encode(query)

# We use cosine-similarity and torch.topk to find the highest 2 scores
cos_scores = cos_sim(query_embedding, sentence_embeddings)[0]
top_results = torch.topk(cos_scores, k=2)

for i in range(2):
  print(f"{float(top_results.values[i]):.{3}}: {sentences[top_results.indices[i]]}")

QUERY:: I love sports
0.275: League: Change or resign.
0.251: Bossi: I'm very happy


### Alternatively,

we can also use util.semantic_search to perform cosine similarty + topk, via scanning the input embeddings (notice that the score is the same as above), taken from ***sentence_transformers***.

In [23]:
hits = util.semantic_search(query_embedding, sentence_embeddings, top_k=2)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print("{:.3f}:".format(hit['score']), sentences[hit['corpus_id']])


0.275: League: Change or resign.
0.251: Bossi: I'm very happy


## **Brute-force scan via IndexFlatL2 (L2 distance)**

IndexFlatL2 measures the L2 distance between all the vectors loaded into the index and our query vector.

It’s simple, very accurate, but not too fast.


In [24]:
d = sentence_embeddings.shape[1]
n = sentence_embeddings.shape[0] ## Also:: len(sentence_embeddings)

# Build the index
index = faiss.IndexFlatL2(d) # initialize
index.add(sentence_embeddings) # add embeddings

print(f"Dimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# Then search given a query `xq` and number of nearest neigbors to return `k`.
# The query can be a list of phrases, here just one
print("\nQUERY: {}\n\n".format(query))
xq = model.encode([query])
k=4

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
#
# They are matrices because we can input more query sentences via xq
#
start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(k):
  print(f"sentence: {sentences[I[0][i]]}")
  print(f"Index: {I[0][i]}, with distance: {D[0][i]:.2f}\n")


Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True


QUERY: I love sports


Elapsed time:  0.41 millisecs

sentence: League: Change or resign.
Index: 295, with distance: 1.45

sentence: Bossi: I'm very happy
Index: 28, with distance: 1.50

sentence: Green light to 1 billion for the Olympic Games in Milan and Cortina 2026 and for the Ryder Cup of golf
Index: 100, with distance: 1.51

sentence: No one's hurt.
Index: 204, with distance: 1.59



## **Brute-force scan via IndexFlatIP (inner-product distance)**


In [25]:
d = sentence_embeddings.shape[1]
n = len(sentence_embeddings)

index = faiss.IndexFlatIP(d) # Initialize
index.add(sentence_embeddings) # Add embeddings
print(f"Dimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# Then search given a query `xq` and number of nearest neigbors to return `k`.
# The query can be a list of phrases, here just one
print("\nQUERY: {}\n\n".format(query))
xq = model.encode([query])
k=4

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(k):
  print(f"sentence: {sentences[I[0][i]]}")
  print(f"Index: {I[0][i]}, with distance: {D[0][i]:.2f}\n")


Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True


QUERY: I love sports


Elapsed time:  0.37 millisecs

sentence: League: Change or resign.
Index: 295, with distance: 0.28

sentence: Bossi: I'm very happy
Index: 28, with distance: 0.25

sentence: Green light to 1 billion for the Olympic Games in Milan and Cortina 2026 and for the Ryder Cup of golf
Index: 100, with distance: 0.24

sentence: No one's hurt.
Index: 204, with distance: 0.21



# So, how can we make our search faster?

There are two primary approaches:

*   **Reduce search scope** — we can do this by clustering or organizing vectors into tree structures based on certain attributes, similarity, or distance — and restricting our search to closest clusters or filter through most similar branches.

*   **Reduce vector size** — through dimensionality reduction or reducing the number of its representing our vectors values.

Using either of these approaches means that we are no longer performing an exhaustive nearest-neighbors search but an approximate nearest-neighbors (ANN) search — as we no longer search the entire, full-resolution dataset.



# **CLUSTERING: reduce the search scope**

We can imagine to have ***nlist*** centroids and that all our embeddings vectors as each being contained within a Voronoi cell.

When we introduce a new query vector, we first measure its distance with respect to the centroids, then restrict our search scope to that centroid’s cell and use IndexFlatL2 inside it (this index is used as a sort of "*quantizer*").

The index has many parameters that can be fine-tuned to our specific accuracy/speed requirements.

In [26]:
#######################################
#
# Voronoi diagrams: just one probe
#
#######################################

nlist = 20  # how many cells to form, they must be less than #vectors
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist) # We add clustering

index.train(sentence_embeddings)
index.add(sentence_embeddings)
print(f"Dimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# Then search given a query `xq` and number of nearest neigbors to return `k`.
# The query can be a list of phrases, here just one
print("\nQUERY: {}\n\n".format(query))
xq = model.encode([query])


# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
k=4
start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(k):
  print(f"sentence: {sentences[I[0][i]]}")
  print(f"Index: {I[0][i]}, with distance: {D[0][i]:.2f}\n")


Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True


QUERY: I love sports


Elapsed time:  0.32 millisecs

sentence: League: Change or resign.
Index: 295, with distance: 1.45

sentence: League leader replies: “The fight against drugs should unite”
Index: 330, with distance: 1.66

sentence: Participants invited to bring a musical instrument to create a great “fish orchestra”
Index: 363, with distance: 1.76

sentence: They also give them new shoes to continue the journey.
Index: 383, with distance: 1.79



We can improve the accuracy of ***IndexIVFFlat*** by increasing the search scope.

We do this by increasing the ***nprobe*** attribute value — which defines how many nearby cells to search. Even with the larger *nprobe* value we still see much faster responses than we returned with our IndexFlatL2-only index.

The index has many parameters that can be fine-tuned to our specific accuracy/speed requirements.

In [27]:
#######################################
#
# Voronoi diagrams: Many probes
#
#######################################

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
index.nprobe = 3 # must be less than nlist
k=4

print("\nQUERY: {}\n\n".format(query))
xq = model.encode([query])
start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(k):
  print(f"sentence: {sentences[I[0][i]]}")
  print(f"Index: {I[0][i]}, with distance: {D[0][i]:.2f}\n")



QUERY: I love sports


Elapsed time:  0.22 millisecs

sentence: League: Change or resign.
Index: 295, with distance: 1.45

sentence: In the Town Hall square the event of the League.
Index: 355, with distance: 1.61

sentence: League leader replies: “The fight against drugs should unite”
Index: 330, with distance: 1.66

sentence: In the parade-event the 50 years of career were revived: from sailor looks to cone bras, to dark clothes and mechanical creatures.
Index: 319, with distance: 1.71



# **Locality Sensitive Hashing: reduce search scope**

By increasing the number of allocated ***nbits*** (up to 64), you increase the recall but you slowdown the query.


In [28]:
# resolution of bucketed vectors: with *64 we get up to 90% recall
nbits = d*4

# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.train(sentence_embeddings)
index.add(sentence_embeddings)
print(f"Dimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
k=4

print("\nQUERY: {}\n\n".format(query))
xq = model.encode([query])

start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(4):
  print(f"nbits {nbits}, sentence {I[0][i]}: {sentences[I[0][i]]}")


##
## Change resolution of bucketed vectors
##
nbits = d*8

# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.train(sentence_embeddings)
index.add(sentence_embeddings)
print(f"\n\nDimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
k=4

xq = model.encode([query])

start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(4):
  print(f"nbits {nbits}, sentence {I[0][i]}: {sentences[I[0][i]]}")


Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True


QUERY: I love sports


Elapsed time:  1.01 millisecs

nbits 1536, sentence 100: Green light to 1 billion for the Olympic Games in Milan and Cortina 2026 and for the Ryder Cup of golf
nbits 1536, sentence 295: League: Change or resign.
nbits 1536, sentence 28: Bossi: I'm very happy
nbits 1536, sentence 355: In the Town Hall square the event of the League.


Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True

Elapsed time:  1.28 millisecs

nbits 3072, sentence 295: League: Change or resign.
nbits 3072, sentence 100: Green light to 1 billion for the Olympic Games in Milan and Cortina 2026 and for the Ryder Cup of golf
nbits 3072, sentence 28: Bossi: I'm very happy
nbits 3072, sentence 330: League leader replies: “The fight against drugs should unite”


# **QUANTIZATION: reduce the vector size**

Faiss comes with the ability to compress our vectors using Product Quantization (PQ). PQ approximates the distance/similarity calculation instead, by compressing the vectors themselves, which consists of three steps:

1.   We split the original vector into several subvectors.

2.   For each set of subvectors, we perform a clustering operation — creating multiple centroids for each sub-vector set.

3. We replace each sub-vector with the ID of it’s nearest set-specific centroid.

To implement it, we use the IndexIVFPQ index — we’ll also need to train the index before adding our embeddings.

In [29]:
#######################################
#
# Quantization
#
#######################################

m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

index.train(sentence_embeddings)
index.add(sentence_embeddings)
print(f"Dimensions: {d}, number of vectors: {n}, number of indexed vectors: {index.ntotal}, trained: {index.is_trained}\n")

# I = matrix of indexes of the k most similar vectors for each xq's phrase
# D = matrix of squared Euclidean distances for each result vector for xq
k=4

index.nprobe = 3  # align to previous IndexIVFFlat nprobe value

xq = model.encode([query])

start = time.time()
D, I = index.search(xq, k)
end = time.time() #in secs
print(f"Elapsed time: {1000 * (end - start): .2f} millisecs\n")

for i in range(4):
  print(f"sentence {I[0][i]}, with distance: {D[0][i]:.2f}: {sentences[I[0][i]]}\n")



Dimensions: 384, number of vectors: 400, number of indexed vectors: 400, trained: True

Elapsed time:  0.41 millisecs

sentence 295, with distance: 1.36: League: Change or resign.

sentence 319, with distance: 1.42: In the parade-event the 50 years of career were revived: from sailor looks to cone bras, to dark clothes and mechanical creatures.

sentence 320, with distance: 1.44: Lots of VIPs present.

sentence 363, with distance: 1.45: Participants invited to bring a musical instrument to create a great “fish orchestra”

